In [1]:
import os, shlex
from subprocess import check_output
repo_rootdir = check_output(shlex.split("git rev-parse --show-toplevel")).strip().decode('ascii')
os.chdir(repo_rootdir)

from random import shuffle
import numpy as np
import ipywidgets as widgets
import yaml
from copy import deepcopy
import json
import re
import itertools
import numpy as np
import scipy

from scipy.spatial import procrustes as procrustes

import glob, os, json
import pandas as pd

#from VTKHelpers.VTKMesh import VTKObject as Mesh
#from cardiac_mesh import CardiacMesh
from sklearn.decomposition import PCA, IncrementalPCA as IPCA
import pickle as pkl
#from config.config_parser import read_default_config

In [2]:
from data.DataModules import CardiacMeshPopulationDataset
from config.cli_args import CLI_args, overwrite_config_items
from config.load_config import load_yaml_config, to_dict

In [3]:
ref_config = load_yaml_config("config_files/config.yaml")

In [4]:
MESH_PKL = "/home/user/01_repos/CardiacCOMA/data/LVED_all_Rodrigos_segmentation_60728_subjects.pkl"
meshes = pkl.load(open(MESH_PKL, "rb"))

In [5]:
CARDIAC_COMA_REPO = "/home/user/01_repos/CardiacCOMA/"

In [6]:
PROCRUSTES_REPLICATION_FILE = f"{CARDIAC_COMA_REPO}/data/procrustes_transforms_LVED_all_Rodrigos_segmentation_60728_subjects.pkl"
procrustes_transforms = pkl.load(open(PROCRUSTES_REPLICATION_FILE, "rb")) 

In [7]:
meshes_dataset = CardiacMeshPopulationDataset(meshes, procrustes_transforms)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60728/60728 [01:16<00:00, 789.52it/s]


In [8]:
N = 5000

In [9]:
shuffled_ids = [x for x in range(len(meshes_dataset))]
shuffle(shuffled_ids)

In [10]:
ids = [meshes_dataset[i]["id"] for i, _ in enumerate(meshes_dataset)]
meshes_lst = [meshes_dataset[id]["s"].numpy() for id in ids]
meshes_np = np.array(meshes_lst)
flattened_meshes = meshes_np.reshape(len(meshes_dataset),-1)
output_dir = "output/PCA__LV__%s_samples" % N
flattened_meshes[shuffled_ids[:N],:].shape

(5000, 15660)

Center meshes around mean shape:

In [11]:
# flattened_meshes -= flattened_meshes.mean(0)

In [12]:
perf_dfs = []
n_comps = 16

# meshes_lst, ids_lst = flattened_partition(cardiac_data, config["nTraining"])
# print(shuffled_ids)

# for n in range(1, n_comps+1):
for n in [16]:
  
    print(n)
    
    #TODO: This doesn't have to be done again for each n, but I need to figure out how to do it.
    training_meshes = flattened_meshes[shuffled_ids[:N],:]
    pp = PCA(n).fit(training_meshes)    
      
    meshes_reduced = pp.transform(flattened_meshes)
  
    meshes_reconstructed = pp.inverse_transform(meshes_reduced)
    mse = ((flattened_meshes - meshes_reconstructed)**2).mean(axis=1)
    
    mse_shuffled = ((flattened_meshes - meshes_reconstructed[shuffled_ids,:])**2).mean(axis=1)
      
    perf_df = pd.DataFrame({
      "model_id": ["PCA__LV__{}_comps__{}_samples".format(str(n_comps), str(N))] * len(meshes_dataset),
      "subject_id": shuffled_ids,
      "subset": ["training"] * N + ["test"] * (len(meshes_dataset) - N),
      "mse": mse,
      "mse_shuffled": mse_shuffled,
      "n": [n] * len(meshes_dataset),      
    })
    
    perf_df.index = shuffled_ids
    perf_dfs.append(perf_df)
          

16


In [13]:
pd.concat(perf_dfs).to_csv("PCA__LV__16_comps__5000_samples__performance.csv", header=True, index=False)

In [17]:
flattened_meshes

array([[ 24.642893  ,   4.01576   ,  27.396309  , ...,   9.306356  ,
         -2.800227  , -52.41445   ],
       [ 26.53488   ,   3.5433652 ,  29.448126  , ...,   8.243105  ,
         -1.5773598 , -55.81192   ],
       [ 30.64968   ,   4.547135  ,  33.65808   , ...,  11.235202  ,
          0.24474011, -61.79597   ],
       ...,
       [ 31.036776  ,   5.254566  ,  34.53359   , ...,   8.929912  ,
         -1.4252867 , -56.966278  ],
       [ 29.542198  ,   5.0099645 ,  31.39196   , ...,  10.54889   ,
         -3.3885365 , -58.265244  ],
       [ 27.969158  ,   3.9376342 ,  30.323282  , ...,   9.623464  ,
         -2.5451443 , -56.829796  ]], dtype=float32)

In [16]:
meshes_reduced

array([[ 164.029     ,  -31.030481  ,  -74.24229   , ...,   -6.062747  ,
          -2.4639282 ,    0.90578544],
       [  44.65998   ,    2.964386  , -119.57689   , ...,   -4.0863748 ,
           4.7335596 ,   -1.3913312 ],
       [-292.851     ,  -52.095676  ,  -17.993368  , ...,    3.103015  ,
          -0.92941463,   -1.4097089 ],
       ...,
       [-154.68541   ,   33.485813  ,   -4.7383013 , ...,   -7.155405  ,
          21.781952  ,    0.7999135 ],
       [-220.9974    ,  -15.743806  ,   48.544895  , ...,   -3.6189575 ,
           2.4747715 ,    7.8488345 ],
       [-110.51882   ,  -40.622166  ,   10.440153  , ...,  -18.149647  ,
           3.2030146 ,    5.80313   ]], dtype=float32)

In [19]:
z_df = pd.DataFrame(meshes_reduced)
z_df.columns = ["z" + str(z).zfill(3) for z in z_df.columns]
z_df["ID"] = list(np.array(ids))# [shuffled_ids])
z_df.to_csv("PCA__LV__16_comps__5000_samples__latent_space.csv", header=True, index=False)